# Data Job User Agregation

Notebook de desenvolvimento responsável por realizar agregação dos dados por usuário que utilizam um aplicativo móvel


<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/Foiac/MobileFraudDetectSolution/main/Editaveis/Imagens/goldjobtransformer.png" alt="Clean Data" style="width: 800px">
</div>

#### Import dependecies

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

#### Path Definitions

In [0]:
storage_account_name  = "stacmfraud"
container_name = "cont-fraud"

database_name_input = "silver_mobile"
table_name_input = "tab_mobil_access"

database_name_output = "gold_mobile"
table_name_output = "tabl_fraud_indi"
container_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/gold/{database_name_output}"
delta_table_path_output = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/gold/{database_name_output}/{table_name_output}"

#### Database Create

In [0]:
spark.sql(f"""CREATE DATABASE IF NOT EXISTS {database_name_output} LOCATION '{container_path}'""")

spark.sql(f"""CREATE TABLE IF NOT EXISTS {database_name_output}.{table_name_output} (
    UID STRING,
    ATTEMPT_ACCESS INTEGER,
    SUCCESS_ACCESS INTEGER,
    DEVICES INTEGER,
    NETWORKS INTEGER,
    PASSWORDS INTEGER,
    TRANSACTIONS_DEVICES INTEGER,
    APP_VERSIONS INTEGER,
    `LOCATIONS` INTEGER,
    RISK_FLAG INTEGER,
    DAT_REF STRING
    ) 
USING DELTA
LOCATION '{delta_table_path_output}'""")


DataFrame[]

#### Read Databricks Catalog Table

In [0]:
filter_date = '2024-11-01'
df = spark.table(f"{database_name_input}.{table_name_input}").filter(F.col("DAT_REF") == filter_date)
display(df)

### Rules to Business Metrics

- Access attempt per device
- Users Access Success 
- Devices per _User_
- Networks per _User_
- Passwords per _User_
- Last Transaction State
- Aplication versions per _User_
- Locations per _User_

In [0]:
df_agg = (df
             .select("UID", "ERROR_INF", "IMEI", "NETWORK", "PASSWORD", "TRANSACTION", "APP_VERSION", "LATITUDE", "LONGITUDE")
             .groupBy("UID")
             .agg(
               F.count("*").cast(IntegerType()).alias("ATTEMPT_ACCESS"),
               F.sum(
                 F.when(F.col("ERROR_INF")=="Oper. com Sucesso", 1).otherwise(0)
                 ).cast(IntegerType()).alias("SUCCESS_ACCESS"),
               F.countDistinct("IMEI").cast(IntegerType()).alias("DEVICES"),
               F.countDistinct("NETWORK").cast(IntegerType()).alias("NETWORKS"),
               F.countDistinct("PASSWORD").cast(IntegerType()).alias("PASSWORDS"),
               F.countDistinct(F.when(F.col("TRANSACTION") == True, F.col("IMEI"))).cast(IntegerType()).alias("TRANSACTIONS_DEVICES"),
               F.countDistinct("APP_VERSION").cast(IntegerType()).alias("APP_VERSIONS"),
               F.countDistinct("LATITUDE", "LONGITUDE").cast(IntegerType()).alias("LOCATIONS")
             )
             )

display(df_agg)

### Rules to Risk `Flag`

In [0]:
df_gold = (df_agg
           .withColumn("RISK_FLAG", 
                       F.when((( F.col("ATTEMPT_ACCESS") > 50 ) | 
                               ( F.col("TRANSACTIONS_DEVICES") > 4 ) | 
                               ( F.col("PASSWORDS") > 4 ) |
                               ( F.col("LOCATIONS") > 100 )
                               ), 1)
                       .otherwise(0)
                       .cast(IntegerType())
           )
           .withColumn("DAT_REF", F.lit(filter_date))
)
display(df_gold)

### Write Data on `Gold Table` 

In [0]:
df_gold.write \
    .format("delta") \
    .mode("append") \
    .option("path", delta_table_path_output) \
    .saveAsTable(f"{database_name_output}.{table_name_output}")